In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!git clone -q https://github.com/dvolynov/HEA_network.git

In [ ]:
%cd HEA_network

In [ ]:
!pip -q install -r 'requirements.txt'

In [ ]:
from modules.initializer import Initializer
from modules.transition_matrix import TransitionMatrix
from modules.config import Config

In [ ]:
%cd '../'

In [ ]:
data2 = pd.read_csv('./clear_out_data.csv')

In [ ]:
data2

In [ ]:
data2_1 = data2.iloc[0*round(len(data2)/5):1*round(len(data2)/5)]
data2_2 = data2.iloc[1*round(len(data2)/5):2*round(len(data2)/5)]
data2_3 = data2.iloc[2*round(len(data2)/5):3*round(len(data2)/5)]
data2_4 = data2.iloc[3*round(len(data2)/5):4*round(len(data2)/5)]
data2_5 = data2.iloc[4*round(len(data2)/5):5*round(len(data2)/5)]

In [ ]:
from modules.initializer import Initializer
from modules.transition_matrix import TransitionMatrix
from modules.config import Config
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [ ]:
part = 5

In [ ]:
decomp2_file = open(f'./decomp2_{part}_text.txt','w')
decomp2_file.close()
decomp_file = open(f'./decomp_{part}_text.txt','w')
decomp_file.close()
system_file = open(f'./system_text_{part}.txt','w')
system_file.close()

In [ ]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from modules.initializer import Initializer
from modules.transition_matrix import TransitionMatrix
from modules.config import Config
import pandas as pd

# Assuming data2 is already loaded as a pandas DataFrame
def process_formula(value, k=0):
    if k == 0:
        formula = value.replace('-', '1')
    else:
        formula = value
    config = Config(formula)
    initializer = Initializer(config)
    atoms = initializer.distribute(end=1000, step=200)
    matrix = TransitionMatrix(atoms, config).data
    return matrix

def process_row(row):
    try:
        index, i = row
    #     if index % 500 == 0:
    #         print(index)
        system = i[1]
        decomp = eval(i[10])
        decomp2 = eval(i[11])

        # Process system
        system_matrix = process_formula(system)
        string_elements = map(str, system_matrix.flatten())
        resulting_string = ', '.join(string_elements)

        # Process decomp
        decomp_sum_matrix = np.zeros((40,40))
        for j in decomp:
            decomp_matrix = process_formula(j, k=1)
            decomp_sum_matrix += decomp_matrix
        string_elements2 = map(str, decomp_sum_matrix.flatten())
        resulting_string2 = ', '.join(string_elements2)

        # Process decomp2
        decomp2_sum_matrix = np.zeros((40,40))
        for m in decomp2:
            decomp2_matrix = process_formula(m, k=1)
            decomp2_sum_matrix += decomp2_matrix
        string_elements3 = map(str, decomp2_sum_matrix.flatten())
        resulting_string3 = ', '.join(string_elements3)

        return resulting_string, resulting_string2, resulting_string3
    except:
        print(row)
        return 'error', 'error', 'error'

total = len(data2_5)

with tqdm(total=total) as pbar:
    with ProcessPoolExecutor(max_workers=8) as executor:
        results = list(tqdm(executor.map(process_row, data2_5.iterrows()), total=total))
        for res in results:
            resulting_string, resulting_string2, resulting_string3 = res
            
            # Write results to files
            with open(f'./decomp2_{part}_text.txt', 'a') as file:
                file.write(resulting_string + '\n')
            with open(f'./decomp_{part}_text.txt', 'a') as file:
                file.write(resulting_string2 + '\n')
            with open(f'./system_text_{part}.txt', 'a') as file:
                file.write(resulting_string3 + '\n')
            pbar.update(1)


In [ ]:
part = 3

In [ ]:
with open(f'./system_text_{part}.txt', 'r') as fp:
    data = fp.readlines()

In [ ]:
[round(float(i), 5) for i in data[1].split(',')]

In [ ]:
for i in data:
    if i [0] == 'e':
        print(i)

In [ ]:
X = []
Y = []

In [ ]:
for index in range(1,6):
    print(index)
    with open(f'./system_text_{index}.txt', 'r') as fp:
        x = fp.readlines()
    
    for i in x:
        if i != 'error\n':
            X.append([round(float(j), 5) for j in i.split(',')])
        else:
            continue
    
    with open(f'./decomp_{index}_text.txt', 'r') as fp1:
        y = fp1.readlines()
    
    for k in y:
        if k != 'error\n':
            Y.append([round(float(z), 5) for z in k.split(',')])
        else:
            continue
    
    fp.close()
    fp1.close()

In [ ]:
import pickle

# with open('./X_train.pkl', 'wb') as file: 
#     pickle.dump(Y, file) 

In [ ]:
with open('./Y_arr_train.pkl', 'rb') as file: 
    Y = pickle.load(file) 

In [ ]:
Y[0]

In [ ]:
import numpy as np
Y_arr=np.array([np.array(xi) for xi in Y])

In [ ]:
with open('./Y_arr_train.pkl', 'wb') as file: 
    pickle.dump(Y_arr, file) 

In [ ]:
len(X)

In [ ]:
import pickle
import numpy as np

In [ ]:
with open('./Y_arr_train.pkl', 'rb') as file: 
    Y = pickle.load(file)

In [ ]:
with open('./X_arr_train.pkl', 'rb') as file: 
    X = pickle.load(file) 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, InputLayer

# Input shape
input_shape = 1600  # 40x40 matrix
input_dim = 1600  # Flattened size

# Create the model
model = Sequential()

# Input layer
model.add(InputLayer(input_shape))

# Hidden layer
model.add(Dense(6400, activation='relu'))

# Output layer
model.add(Dense(input_dim, activation='linear'))
# model.add(Reshape(input_shape))

# Compile the model
model.compile(optimizer='adam', loss='mse')


In [ ]:
# Example training data
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32)


In [ ]:
# Display the model summary
model.summary()

In [ ]:
len(Y[0])